In [1]:
# Install the required libraries
#SAM
!pip install git+https://github.com/facebookresearch/segment-anything.git
#Transformers
!pip install -q git+https://github.com/huggingface/transformers.git
#Datasets to prepare data and monai if you want to use special loss functions
!pip install datasets
!pip install -q monai
#Patchify to divide large images into smaller patches for training. (Not necessary for smaller images)
!pip install patchify

  Cloning https://github.com/facebookresearch/segment-anything.git to c:\users\admin\appdata\local\temp\pip-req-build-7nrmdj9f
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for segment-anything: started
  Running setup.py install for segment-anything: finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git 'C:\Users\Admin\AppData\Local\Temp\pip-req-build-7nrmdj9f'
  DEPRECATION: segment-anything is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


^C
     -------------------------------------- 542.0/542.0 kB 4.3 MB/s eta 0:00:00
     -------------------------------------- 116.3/116.3 kB 7.1 MB/s eta 0:00:00
  Using cached huggingface_hub-0.22.2-py3-none-any.whl (388 kB)
     ------------------------------------- 370.7/370.7 kB 22.5 MB/s eta 0:00:00
  Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl (78 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl (145 kB)
  Using cached filelock-3.13.4-py3-none-any.whl (11 kB)
     ---------------------------------------- 25.9/25.9 MB 8.3 MB/s eta 0:00:00
     -------------------------------------- 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 50.4/50.4 kB ? eta 0:00:00
     ---------------------------------------- 76.4/76.4 kB 4.4 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


: 

: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import os
from patchify import patchify  #Only to handle large images
import random
from scipy import ndimage
import cv2
from datasets import Dataset
from PIL import Image
import matplotlib.pyplot as plt
import random
import torch

In [ ]:
# Đường dẫn tới thư mục chứa các hình ảnh
image_folder = "/kaggle/input/otu-2d/OTU_2D/test/Image"

# Lấy danh sách các tệp trong thư mục và sắp xếp theo thứ tự tên tăng dần từ A đến Z
image_files = sorted(os.listdir(image_folder))

# Khởi tạo một danh sách để chứa các hình ảnh dưới dạng mảng NumPy
image_array_list = []

# Lặp qua tất cả các tệp trong thư mục đã sắp xếp
for filename in image_files:
    # Kiểm tra xem tệp có phải là hình ảnh không
    if filename.endswith(('.JPG')):
        # Đường dẫn đầy đủ tới hình ảnh
        img_path = os.path.join(image_folder, filename)

        # Đọc hình ảnh
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (256, 256))

        # Kiểm tra xem hình ảnh có được đọc thành công hay không
        if img is not None:
            # Thêm hình ảnh vào danh sách
            image_array_list.append(img)
        else:
            print(f"Không thể đọc hình ảnh {filename}")

# Chuyển đổi danh sách hình ảnh thành mảng NumPy
test_images_np = np.array(image_array_list)

# In ra kích thước của mảng hình ảnh
print("Kích thước của mảng hình ảnh:", test_images_np.shape)

In [ ]:
# Đường dẫn tới thư mục chứa các hình ảnh
image_folder = "/kaggle/input/otu-2d/OTU_2D/test/Label"

# Lấy danh sách các tệp trong thư mục và sắp xếp theo thứ tự tên tăng dần từ A đến Z
image_files = sorted(os.listdir(image_folder))

# Khởi tạo một danh sách để chứa các hình ảnh dưới dạng mảng NumPy
image_array_list = []

# Lặp qua tất cả các tệp trong thư mục đã được sắp xếp
for filename in image_files:
    # Kiểm tra xem tệp có phải là hình ảnh không
    if filename.endswith('.PNG'):
        # Đường dẫn đầy đủ tới hình ảnh
        img_path = os.path.join(image_folder, filename)

        # Đọc hình ảnh
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (256, 256))
        img = img / 255.0
        (thresh, img) = cv2.threshold(img, 0, 1, cv2.THRESH_BINARY)

        # img = np.int32(img)

        # Kiểm tra xem hình ảnh có được đọc thành công hay không
        if img is not None:
            # Thêm hình ảnh vào danh sách
            image_array_list.append(img)
        else:
            print(f"Không thể đọc hình ảnh {filename}")

# Chuyển đổi danh sách hình ảnh thành mảng NumPy
test_labels_np = np.array(image_array_list)

# In ra kích thước của mảng hình ảnh
print("Kích thước của mảng mặt nạ:", test_labels_np.shape)

In [ ]:
# Convert the NumPy arrays to Pillow images and store them in a dictionary
dataset_dict = {
    "image": [Image.fromarray(img).convert('RGB') for img in test_images_np],
    "label": [Image.fromarray(mask).convert('I') for mask in test_labels_np],
}

# Create the dataset using the datasets.Dataset class
test_dataset = Dataset.from_dict(dataset_dict)
test_dataset_image = Dataset.from_dict(dataset_dict)

In [ ]:
test_dataset

In [ ]:
test_dataset_image

In [ ]:
#Get bounding boxes from mask.
def get_bounding_box(ground_truth_map):
  # get bounding box from mask
  y_indices, x_indices = np.where(ground_truth_map > 0)
  x_min, x_max = np.min(x_indices), np.max(x_indices)
  y_min, y_max = np.min(y_indices), np.max(y_indices)
  # add perturbation to bounding box coordinates
  H, W = ground_truth_map.shape
  x_min = max(0, x_min - np.random.randint(0, 20))
  x_max = min(W, x_max + np.random.randint(0, 20))
  y_min = max(0, y_min - np.random.randint(0, 20))
  y_max = min(H, y_max + np.random.randint(0, 20))
  bbox = [x_min, y_min, x_max, y_max]

  return bbox

In [ ]:
from torch.utils.data import Dataset

class SAMDataset(Dataset):
  """
  This class is used to create a dataset that serves input images and masks.
  It takes a dataset and a processor as input and overrides the __len__ and __getitem__ methods of the Dataset class.
  """
  def __init__(self, dataset, processor):
    self.dataset = dataset
    self.processor = processor

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    item = self.dataset[idx]
    image = item["image"]
    ground_truth_mask = np.array(item["label"])

    # get bounding box prompt | vẽ box cho mặt nạ
    prompt = get_bounding_box(ground_truth_mask)

    # prepare image and prompt for the model | Chuẩn bị mặt nạ và hộp giới hạn
    inputs = self.processor(image, input_boxes=[[prompt]], return_tensors="pt")

    # remove batch dimension which the processor adds by default | Loại bỏ chiều Batch được thêm vào mặc định
    inputs = {k:v.squeeze(0) for k,v in inputs.items()}

    # add ground truth segmentation | Thêm ground truth để đánh giá việc Segment sau này, đánh giá hiệu suất mô hình
    inputs["ground_truth_mask"] = ground_truth_mask

    return inputs

In [ ]:
# Initialize the processor
from transformers import SamProcessor
processor_SAM = SamProcessor.from_pretrained("facebook/sam-vit-base")
processor_MedSAM = SamProcessor.from_pretrained("wanglab/medsam-vit-base")

In [ ]:
test_dataset_SAM = SAMDataset(dataset=test_dataset, processor=processor_SAM)
test_dataset_MedSAM = SAMDataset(dataset=test_dataset, processor=processor_MedSAM)

In [ ]:
example = test_dataset_SAM[0]
for k,v in example.items():
  print(f'{k}: {v.shape}')

In [ ]:
from torch.utils.data import DataLoader
test_dataloader_SAM = DataLoader(test_dataset_SAM, batch_size=2, shuffle=True, drop_last=False)
test_dataloader_MedSAM = DataLoader(test_dataset_MedSAM, batch_size=2, shuffle=True, drop_last=False)

In [ ]:
batch = next(iter(test_dataloader_SAM))
for k,v in batch.items():
  print(k,v.shape)

In [ ]:
batch = next(iter(test_dataloader_MedSAM))
for k,v in batch.items():
  print(k,v.shape)

In [ ]:
batch["ground_truth_mask"].shape

In [ ]:
# Load the model
from transformers import SamModel
model_SAM = SamModel.from_pretrained("facebook/sam-vit-base")
model_MedSAM = SamModel.from_pretrained("wanglab/medsam-vit-base")

# make sure we only compute gradients for mask decoder
for name, param in model_SAM.named_parameters():
  if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
    param.requires_grad_(False)

for name, param in model_MedSAM.named_parameters():
  if name.startswith("vision_encoder") or name.startswith("prompt_encoder"):
    param.requires_grad_(False)

In [ ]:
from tqdm import tqdm
from statistics import mean
import torch
from torch.nn.functional import threshold, normalize

smooth=1e-10

# Define functions for calculating evaluation metrics
def dice(predicted, target):
    true_positive = torch.sum(predicted * target)
    false_negative = torch.sum(target) - true_positive
    false_positive = torch.sum(predicted) - true_positive
    return (2. * true_positive + smooth) / (2. *true_positive + false_negative + false_positive + smooth)

def iou(predicted, target):
    intersection = torch.sum(predicted * target)
    union = torch.sum(predicted) + torch.sum(target) - intersection
    return (intersection + smooth) / (union + smooth)

def recall(predicted, target):
    true_positive = torch.sum(predicted * target)
    false_negative = torch.sum(target) - true_positive
    return (true_positive + smooth) / (true_positive + false_negative + smooth)

def precision(predicted, target):
    true_positive = torch.sum(predicted * target)
    false_positive = torch.sum(predicted) - true_positive
    return (true_positive + smooth) / (true_positive + false_positive + smooth)

Test SAM original

In [ ]:
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model_SAM.to(device)

# Tải trọng số từ checkpoint
checkpoint_path = "/kaggle/input/validation-100epochs/checkpoint_SAM/best_model_weights.pt"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Load trọng số vào mô hình
model_SAM.load_state_dict(checkpoint)

# Đặt mô hình vào chế độ đánh giá
model_SAM.eval()

# Tiếp tục quá trình kiểm tra mô hình như đã thực hiện trước đó
test_dice_scores = []
test_iou_scores = []
test_recall_scores = []
test_precision_scores = []

with torch.no_grad():
    for batch in tqdm(test_dataloader_SAM):
        # Forward pass
        outputs = model_SAM(pixel_values=batch["pixel_values"].to(device),
                        input_boxes=batch["input_boxes"].to(device),
                        multimask_output=False)

        # Compute evaluation metrics
        predicted_masks = (torch.sigmoid(outputs['pred_masks']).squeeze() > 0.5).float()
        ground_truth_masks = batch["ground_truth_mask"].float().to(device)

        test_dice_scores.append(dice(predicted_masks, ground_truth_masks))
        test_iou_scores.append(iou(predicted_masks, ground_truth_masks))
        test_recall_scores.append(recall(predicted_masks, ground_truth_masks))
        test_precision_scores.append(precision(predicted_masks, ground_truth_masks))

# Print evaluation metrics
print("\n")
print(f'Test Dice: {torch.tensor(test_dice_scores).mean().item():.4f}')
print(f'Test IOU: {torch.tensor(test_iou_scores).mean().item():.4f}')
print(f'Test Recall: {torch.tensor(test_recall_scores).mean().item():.4f}')
print(f'Test Precision: {torch.tensor(test_precision_scores).mean().item():.4f}')

Test MedSAM original

In [ ]:
# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to the device
model_MedSAM.to(device)

# Tải trọng số từ checkpoint
checkpoint_path = "/kaggle/input/validation-100epochs/checkpoint_SAM/best_model_weights.pt"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Load trọng số vào mô hình
model_MedSAM.load_state_dict(checkpoint)

# Đặt mô hình vào chế độ đánh giá
model_MedSAM.eval()

# Tiếp tục quá trình kiểm tra mô hình như đã thực hiện trước đó
test_dice_scores = []
test_iou_scores = []
test_recall_scores = []
test_precision_scores = []

with torch.no_grad():
    for batch in tqdm(test_dataloader_MedSAM):
        # Forward pass
        outputs = model_MedSAM(pixel_values=batch["pixel_values"].to(device),
                        input_boxes=batch["input_boxes"].to(device),
                        multimask_output=False)

        # Compute evaluation metrics
        predicted_masks = (torch.sigmoid(outputs['pred_masks']).squeeze() > 0.5).float()
        ground_truth_masks = batch["ground_truth_mask"].float().to(device)

        test_dice_scores.append(dice(predicted_masks, ground_truth_masks))
        test_iou_scores.append(iou(predicted_masks, ground_truth_masks))
        test_recall_scores.append(recall(predicted_masks, ground_truth_masks))
        test_precision_scores.append(precision(predicted_masks, ground_truth_masks))

# Print evaluation metrics
print("\n")
print(f'Test Dice: {torch.tensor(test_dice_scores).mean().item():.4f}')
print(f'Test IOU: {torch.tensor(test_iou_scores).mean().item():.4f}')
print(f'Test Recall: {torch.tensor(test_recall_scores).mean().item():.4f}')
print(f'Test Precision: {torch.tensor(test_precision_scores).mean().item():.4f}')

# In hình ảnh 

In [ ]:
import cv2
import numpy as np
from PIL import Image
import torch
import matplotlib.pyplot as plt

# Sử dụng GPU nếu có sẵn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for idx in range(0, len(test_dataset)):  # Duyệt qua các chỉ số từ 1 đến len data
    # load image
    image = test_dataset_image[idx]["image"]

    # get box prompt based on ground truth segmentation map
    ground_truth_mask = np.array(test_dataset_image[idx]["label"])
    prompt = get_bounding_box(ground_truth_mask)

    # prepare image + box prompt for the model
    inputs_SAM = processor_SAM(image, input_boxes=[[prompt]], return_tensors="pt").to(device)
    inputs_MedSAM = processor_MedSAM(image, input_boxes=[[prompt]], return_tensors="pt").to(device)

    # forward pass
    with torch.no_grad():
        outputs_SAM = model_SAM(**inputs_SAM, multimask_output=False)
        outputs_MedSAM = model_MedSAM(**inputs_MedSAM, multimask_output=False)

    # apply sigmoid
    sam_seg_prob = torch.sigmoid(outputs_SAM.pred_masks.squeeze(1))
    # convert soft mask to hard mask
    sam_seg_prob = sam_seg_prob.cpu().numpy().squeeze()
    sam_seg = (sam_seg_prob > 0.5).astype(np.uint8)
    sam_seg = cv2.medianBlur(sam_seg, 7)

    medsam_seg_prob = torch.sigmoid(outputs_MedSAM.pred_masks.squeeze(1))
    # convert soft mask to hard mask
    medsam_seg_prob = medsam_seg_prob.cpu().numpy().squeeze()
    medsam_seg = (medsam_seg_prob > 0.5).astype(np.uint8)
    medsam_seg = cv2.medianBlur(medsam_seg, 7)
    
    fig, axes = plt.subplots(1, 4, figsize=(15, 5))
    # Hiển thị hình ảnh
    axes[0].imshow(image)
    axes[0].set_title("Original")
    axes[0].axis("off")

    # Hiển thị đồ thị cho ground truth mask
    axes[1].imshow(ground_truth_mask)
    axes[1].set_title("Ground truth mask")
    axes[1].axis("off")

    # Hiển thị đồ thị cho predicted mask
    axes[2].imshow(sam_seg)
    axes[2].set_title("SAM Original")
    axes[2].axis("off")

    # Hiển thị đồ thị cho predicted mask
    axes[3].imshow(medsam_seg)
    axes[3].set_title("MedSAM Original")
    axes[3].axis("off")

    plt.show()